In [1]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn import preprocessing
import plotnine as p9


In [2]:
X_train = pd.read_csv("./data/external/train_logs.csv")

X_train = (
    X_train
    .sort_values(["id", "event_id"], ascending=[True, True])
    )

In [3]:
# no explicit record for a pause. pauses are omitted.
X_train['up_time_lag1'] = (
    X_train
    .groupby(['id'])
    ['up_time']
    .shift(1)
    )
X_train['preceding_pause_time'] = (
    X_train['down_time'] - X_train['up_time_lag1']
    )
# expect some negative pause times -- interpret as, no real pause
has_no_real_pause = X_train['preceding_pause_time'] <= 0
X_train.loc[has_no_real_pause, 'preceding_pause_time'] = None

In [4]:
# if pause exceeds threshold duration, a "burst" has ended
MS_PER_S = 1000
SECONDS_PER_BURST = 2

X_train['is_new_burst_start'] = (
    X_train['preceding_pause_time'] > MS_PER_S * SECONDS_PER_BURST
    ).astype(int)
X_train['is_new_burst_start'][0] = 1
X_train['burst_id'] = (
    X_train
    .groupby(['id'])
    ['is_new_burst_start']
    .cumsum()
    )
X_train['burst_time_start'] = (
    X_train
    .groupby(['id', 'burst_id'])
    ['down_time']
    .transform('min')
    )
X_train['burst_time_end'] = (
    X_train
    .groupby(['id', 'burst_id'])
    ['up_time']
    .transform('max')
    )
X_train['burst_duration'] = X_train['burst_time_end'] - X_train['burst_time_start']

/tmp/ipykernel_47659/3022411987.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [5]:
# word count offers a productivity measure
X_train['word_count_lag1'] = (
    X_train
    .groupby(['id'])
    ['word_count']
    .shift(1)
    )

X_train['word_count_delta_event'] = (
    X_train['word_count'] - X_train['word_count_lag1']
    )

X_train['word_count_delta_burst'] = (
    X_train
    .groupby(['id', 'burst_id'])
    ['word_count_delta_event']
    .transform('sum')
    )

In [6]:
# one-way cursor movement might be most productive
# jumping around is choppy
X_train['cursor_position_lag1'] = (
    X_train
    .groupby(['id'])
    ['cursor_position']
    .shift(1)
    )

X_train['has_cursor_position_moved_right'] = (
    X_train['cursor_position'] > X_train['cursor_position_lag1']
    ).astype(int)

In [49]:
# if thoughts are separately clearly by punctuation, writing won't score well
X_train['is_thought_delimiting_punctuation'] = (
    (X_train['text_change'] == ".")
    | (X_train['text_change'] == ",")
    | (X_train['text_change'] == "-")
    | (X_train['text_change'] == "!")
    | (X_train['text_change'] == ";")
    | (X_train['text_change'] == "?")
    ).astype(int)

In [50]:
(
    X_train
    .loc[X_train['is_thought_delimiting_punctuation'] == 1, 'text_change']
    .value_counts()
)

text_change
.    67797
,    54797
-     4492
?     3507
;     2905
!      639
Name: count, dtype: int64

In [7]:
# windows allow for time-sequence features
TOTAL_MIN = 30
SECONDS_PER_MIN = 60
SECONDS_PER_WINDOW = 30

X_train['window_30s'] = pd.cut(
    X_train['down_time'],
    bins=np.arange(
        0, 
        TOTAL_MIN * SECONDS_PER_MIN * MS_PER_S + 5*MS_PER_S*2, 
        SECONDS_PER_WINDOW * MS_PER_S
        )
    )

In [8]:
# summarize pause distr
MS_IN_PAUSE_BUCKET_MAX = 200e3
PAUSE_BUCKET_STEP_MS = 500

X_train['preceding_pause_time_bucket'] = pd.cut(
    X_train['preceding_pause_time'],
    bins=np.arange(
        0, 
        MS_IN_PAUSE_BUCKET_MAX,
        PAUSE_BUCKET_STEP_MS
        )
    )

X_train['preceding_pause_time_bucket'].value_counts()

# WARNING: this representation of pause distribution is dense & large
# a few parameters from distribution model far more succinct


preceding_pause_time_bucket
(0.0, 500.0]            5260183
(500.0, 1000.0]          356216
(1000.0, 1500.0]         126117
(1500.0, 2000.0]          62487
(2000.0, 2500.0]          37948
                         ...   
(192000.0, 192500.0]          1
(175000.0, 175500.0]          0
(194500.0, 195000.0]          0
(194000.0, 194500.0]          0
(184500.0, 185000.0]          0
Name: count, Length: 399, dtype: int64

In [9]:
ACTIVITY_CATEGORIES = ['Nonproduction', 'Input', 'Remove/Cut', 'Replace', 'Paste']

pipeline_activity_onehot = ColumnTransformer(
    transformers=[(
        'onehot_encode', 
        preprocessing.OneHotEncoder(
            categories=[ ACTIVITY_CATEGORIES ], 
            sparse=False, 
            handle_unknown='infrequent_if_exist'
            ),
        ["activity"]
    )],
    remainder='passthrough',
    verbose_feature_names_out=False
    )
pipeline_activity_onehot.fit(X_train)
original_categorical = X_train['activity']

X_train_dtypes = X_train.dtypes.to_dict()
X_train = pipeline_activity_onehot.transform(X_train)
X_train = pd.DataFrame(X_train, columns=pipeline_activity_onehot.get_feature_names_out())
X_train = pd.concat([X_train, original_categorical], axis=1)
X_train = X_train.astype(X_train_dtypes)

/home/alex/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.


In [10]:
for activity in ACTIVITY_CATEGORIES:

    X_train['burst_action_time_' + activity] = (
        X_train
        .assign(activity_x_event_time = lambda x: x['activity_' + activity] * x.action_time)
        .groupby(['id', 'burst_id'])
        ['activity_x_event_time']
        .transform('sum')
        ).astype(float)
    
X_train['burst_type'] = (
    X_train
    [['burst_action_time_' + activity for activity in ACTIVITY_CATEGORIES]]
    .idxmax(axis=1)
    )
X_train['burst_type'] = (
    X_train['burst_type']
    .str
    .replace("burst_action_time_", "", regex=True)
    )

In [11]:
pipeline_burst_type_onehot = ColumnTransformer(
    transformers=[(
        'onehot_encode', 
        preprocessing.OneHotEncoder(
            categories=[ ACTIVITY_CATEGORIES ], 
            sparse=False, 
            handle_unknown='infrequent_if_exist'
            ),
        ["burst_type"]
    )],
    remainder='passthrough',
    verbose_feature_names_out=False
    )
pipeline_burst_type_onehot.fit(X_train)
original_categorical = X_train['burst_type']

X_train_dtypes = X_train.dtypes.to_dict()
X_train = pipeline_burst_type_onehot.transform(X_train)
X_train = pd.DataFrame(X_train, columns=pipeline_burst_type_onehot.get_feature_names_out())
X_train = pd.concat([X_train, original_categorical], axis=1)
X_train = X_train.astype(X_train_dtypes)

/home/alex/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.


In [12]:
for activity in ACTIVITY_CATEGORIES:

    X_train['is_new_burst_start_' + activity] = (
        X_train['is_new_burst_start'] * 
        X_train['burst_type_' + activity]
        )

In [13]:
X_train = X_train[[
    "id",
    "event_id",
    "window_30s",
    "burst_id",
    "burst_type",
    "burst_type_Nonproduction",
    "burst_type_Input",
    "burst_type_Remove/Cut",
    "burst_type_Replace",
    "burst_type_Paste",
    "is_new_burst_start",
    "is_new_burst_start_Nonproduction",
    "is_new_burst_start_Input",
    "is_new_burst_start_Remove/Cut",
    "is_new_burst_start_Replace",
    "is_new_burst_start_Paste",
    "burst_time_start",
    "burst_time_end",
    "burst_duration",
    "word_count_delta_burst",

    "down_time",
    "up_time",	
    "action_time",	
    "activity",	
    "activity_Nonproduction",
    "activity_Input",
    "activity_Remove/Cut",
    "activity_Replace",
    "activity_Paste",
    "down_event",	
    "up_event",	
    "text_change",
    "cursor_position",	
    "word_count",

    "word_count_lag1",
    "word_count_delta_event",

    "has_cursor_position_moved_right",
    "up_time_lag1",
    "preceding_pause_time",
    "preceding_pause_time_bucket",

    "burst_action_time_Nonproduction",
    "burst_action_time_Input",
    "burst_action_time_Remove/Cut",
    "burst_action_time_Replace",
    "burst_action_time_Paste"
    ]]


In [28]:
X_train_marginals_sum_wrt_time = (
    X_train
    .groupby('id')
    [
        ['activity_' + x for x in ACTIVITY_CATEGORIES] 
        + ['is_new_burst_start'] 
        + ['is_new_burst_start_' + x for x in ACTIVITY_CATEGORIES]
        + ['word_count_delta_event']
    ]
    .agg(sum)
    )
X_train_marginals_sum_wrt_time['delete_insert_ratio'] = (
    X_train_marginals_sum_wrt_time['activity_Remove/Cut'] / 
    X_train_marginals_sum_wrt_time['activity_Input'] 
    )

In [29]:
X_train_marginals_central_tendency_wrt_time = (
    X_train
    .groupby('id')
    .agg(
        pause_time_p50 = ('preceding_pause_time', np.median),
        has_cursor_position_moved_right_mean = ('has_cursor_position_moved_right', 'mean'),
        burst_duration_mean = ('burst_duration', 'mean'),
        burst_duration_p50 = ('burst_duration', np.median),
        word_count_delta_burst_p50 = ('word_count_delta_burst', np.median)
        )
    )

In [30]:
X_train_marginals_extremes_wrt_time = (
    X_train
    .groupby('id')
    .agg(
        pause_time_max=('preceding_pause_time', 'max'),
        # approximation to, next longest pause after first long planning pause
        pause_time_p99=('preceding_pause_time', lambda x: x.quantile(0.99)),
        burst_duration_max=('burst_duration', 'max'),
        total_time=('up_time', 'max')
        )
    )

In [31]:
from scipy.stats import lognorm

pause_distr_summary_subjects = []

for X_train_subject in [x for _, x in X_train.groupby('id')]:

    shape, location, scale = lognorm.fit(X_train_subject['preceding_pause_time'].dropna())

    pause_distr_summary = pd.DataFrame({
        'pauses_lognorm_shape': [shape], 
        'pauses_lognorm_location': [location],
        'pauses_lognorm_scale': [scale]
        })
    pause_distr_summary.index = [X_train_subject['id'].iloc[0]]
    
    pause_distr_summary_subjects.append(pause_distr_summary)

X_train_marginals_distr_params_wrt_time = pd.concat(pause_distr_summary_subjects, axis=0)

In [32]:
X_train_marginals_wrt_time = pd.merge(
    X_train_marginals_sum_wrt_time, 
    X_train_marginals_central_tendency_wrt_time,
    how='left',
    left_index=True,
    right_index=True
    )

X_train_marginals_wrt_time = pd.merge(
    X_train_marginals_wrt_time, 
    X_train_marginals_extremes_wrt_time,
    how='left',
    left_index=True,
    right_index=True
    )

X_train_marginals_wrt_time = pd.merge(
    X_train_marginals_wrt_time, 
    X_train_marginals_distr_params_wrt_time,
    how='left',
    left_index=True,
    right_index=True
    )

X_train_marginals_wrt_time = (
    X_train_marginals_wrt_time
    .assign(
        keystroke_speed = lambda x: (x.activity_Input + x['activity_Remove/Cut']) / x.total_time,
        writing_speed = lambda x: (x.word_count_delta_event) / x.total_time
        )
    )

In [33]:
X_train_marginals_wrt_time.head()

,activity_Nonproduction,activity_Input,activity_Remove/Cut,activity_Replace,activity_Paste,is_new_burst_start,is_new_burst_start_Nonproduction,is_new_burst_start_Input,is_new_burst_start_Remove/Cut,is_new_burst_start_Replace,...,word_count_delta_burst_p50,pause_time_max,pause_time_p99,burst_duration_max,total_time,pauses_lognorm_shape,pauses_lognorm_location,pauses_lognorm_scale,keystroke_speed,writing_speed
id,,,,,,,,,,,,,,,,,,,,,
001519c8,120.0,2010.0,417.0,7.0,0.0,125,27.0,86.0,12.0,0.0,...,4.0,154136.0,14109.12,24633,1801969,1.600439,0.650173,141.125479,0.001347,0.000142
0022f953,254.0,1938.0,260.0,1.0,1.0,80,27.0,44.0,9.0,0.0,...,14.0,145899.0,17090.30,49815,1788969,1.878518,15.595657,72.145362,0.001229,0.000179
0042269b,175.0,3515.0,439.0,7.0,0.0,77,29.0,45.0,3.0,0.0,...,16.0,153886.0,10248.40,69505,1771669,1.788094,0.857982,67.276374,0.002232,0.000228
0059420b,99.0,1304.0,151.0,1.0,1.0,87,7.0,77.0,3.0,0.0,...,3.0,101690.0,13174.66,30862,1404469,2.045391,0.107647,148.390852,0.001036,0.000147
0075873a,72.0,1942.0,517.0,0.0,0.0,88,6.0,72.0,10.0,0.0,...,6.0,110688.0,15802.64,32851,1662472,1.498881,1.702449,125.475199,0.001479,0.000152


In [34]:
X_train_marginals_wrt_time.isnull().sum()

activity_Nonproduction                  0
activity_Input                          0
activity_Remove/Cut                     0
activity_Replace                        0
activity_Paste                          0
is_new_burst_start                      0
is_new_burst_start_Nonproduction        0
is_new_burst_start_Input                0
is_new_burst_start_Remove/Cut           0
is_new_burst_start_Replace              0
is_new_burst_start_Paste                0
word_count_delta_event                  0
delete_insert_ratio                     0
pause_time_p50                          0
has_cursor_position_moved_right_mean    0
burst_duration_mean                     0
burst_duration_p50                      0
word_count_delta_burst_p50              0
pause_time_max                          0
pause_time_p99                          0
burst_duration_max                      0
total_time                              0
pauses_lognorm_shape                    0
pauses_lognorm_location           

In [35]:
X_train_by_window = (
    X_train
    .groupby(['id', 'window_30s'])
    [
        ['activity_' + x for x in ACTIVITY_CATEGORIES] 
        + ['is_new_burst_start'] 
        + ['is_new_burst_start_' + x for x in ACTIVITY_CATEGORIES]
        + ['word_count_delta_event']
    ]
    .agg(sum)
    .astype(float)
    .reset_index(drop=False)
    )

X_train_by_window['delete_insert_ratio'] = (
    X_train_by_window['activity_Remove/Cut'] / 
    X_train_by_window['activity_Input'] 
    )

X_train_by_window['window_30s_idx'] = X_train_by_window.index

In [36]:
X_train_windows_variation = (
    X_train_by_window
    .drop(columns=['window_30s', 'window_30s_idx'])
    .groupby(['id'])
    .agg(np.std)
    )

X_train_windows_variation.columns = [
    x + "_stddev"
    for x in X_train_windows_variation.columns
    ]

In [37]:
X_train_windows_variation.head()

,activity_Nonproduction_stddev,activity_Input_stddev,activity_Remove/Cut_stddev,activity_Replace_stddev,activity_Paste_stddev,is_new_burst_start_stddev,is_new_burst_start_Nonproduction_stddev,is_new_burst_start_Input_stddev,is_new_burst_start_Remove/Cut_stddev,is_new_burst_start_Replace_stddev,is_new_burst_start_Paste_stddev,word_count_delta_event_stddev,delete_insert_ratio_stddev
id,,,,,,,,,,,,,
001519c8,3.039244,29.025121,8.569605,0.415450,0.000000,1.205449,0.810245,1.079339,0.403376,0.0,0.0,4.470714,NaN
0022f953,13.243952,40.257771,5.589538,0.129099,0.129099,1.216645,0.768556,0.918116,0.360085,0.0,0.0,7.434839,0.699985
0042269b,3.082436,62.033422,10.014382,0.323732,0.000000,1.303082,0.770025,0.815631,0.219784,0.0,0.0,10.229347,NaN
0059420b,5.044967,20.908896,4.192373,0.129099,0.129099,1.267922,0.372425,1.303082,0.219784,0.0,0.0,3.533696,NaN
0075873a,2.434219,34.888669,16.952668,0.000000,0.000000,1.294927,0.302532,1.161530,0.418499,0.0,0.0,6.851450,NaN


In [38]:
X_train_transform = pd.merge(
    X_train_marginals_wrt_time,
    X_train_windows_variation,
    how='left',
    left_index=True,
    right_index=True
    )

In [39]:
X_train_transform

,activity_Nonproduction,activity_Input,activity_Remove/Cut,activity_Replace,activity_Paste,is_new_burst_start,is_new_burst_start_Nonproduction,is_new_burst_start_Input,is_new_burst_start_Remove/Cut,is_new_burst_start_Replace,...,activity_Replace_stddev,activity_Paste_stddev,is_new_burst_start_stddev,is_new_burst_start_Nonproduction_stddev,is_new_burst_start_Input_stddev,is_new_burst_start_Remove/Cut_stddev,is_new_burst_start_Replace_stddev,is_new_burst_start_Paste_stddev,word_count_delta_event_stddev,delete_insert_ratio_stddev
id,,,,,,,,,,,,,,,,,,,,,
001519c8,120.0,2010.0,417.0,7.0,0.0,125,27.0,86.0,12.0,0.0,...,0.415450,0.000000,1.205449,0.810245,1.079339,0.403376,0.0,0.0,4.470714,NaN
0022f953,254.0,1938.0,260.0,1.0,1.0,80,27.0,44.0,9.0,0.0,...,0.129099,0.129099,1.216645,0.768556,0.918116,0.360085,0.0,0.0,7.434839,0.699985
0042269b,175.0,3515.0,439.0,7.0,0.0,77,29.0,45.0,3.0,0.0,...,0.323732,0.000000,1.303082,0.770025,0.815631,0.219784,0.0,0.0,10.229347,NaN
0059420b,99.0,1304.0,151.0,1.0,1.0,87,7.0,77.0,3.0,0.0,...,0.129099,0.129099,1.267922,0.372425,1.303082,0.219784,0.0,0.0,3.533696,NaN
0075873a,72.0,1942.0,517.0,0.0,0.0,88,6.0,72.0,10.0,0.0,...,0.000000,0.000000,1.294927,0.302532,1.161530,0.418499,0.0,0.0,6.851450,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ffb8c745,189.0,3588.0,960.0,2.0,0.0,41,4.0,29.0,8.0,0.0,...,0.181020,0.000000,0.833446,0.311729,0.724666,0.342803,0.0,0.0,26.137463,NaN
ffbef7e5,148.0,2395.0,60.0,1.0,0.0,90,24.0,65.0,1.0,0.0,...,0.129099,0.000000,1.346685,0.905726,0.944057,0.129099,0.0,0.0,6.167987,0.267295
ffccd6fd,126.0,2849.0,88.0,0.0,0.0,87,13.0,71.0,3.0,0.0,...,0.000000,0.000000,1.267142,0.584885,1.091829,0.219784,0.0,0.0,3.477896,0.135533


In [40]:
X_train_transform.isnull().mean()

activity_Nonproduction                     0.000000
activity_Input                             0.000000
activity_Remove/Cut                        0.000000
activity_Replace                           0.000000
activity_Paste                             0.000000
is_new_burst_start                         0.000000
is_new_burst_start_Nonproduction           0.000000
is_new_burst_start_Input                   0.000000
is_new_burst_start_Remove/Cut              0.000000
is_new_burst_start_Replace                 0.000000
is_new_burst_start_Paste                   0.000000
word_count_delta_event                     0.000000
delete_insert_ratio                        0.000000
pause_time_p50                             0.000000
has_cursor_position_moved_right_mean       0.000000
burst_duration_mean                        0.000000
burst_duration_p50                         0.000000
word_count_delta_burst_p50                 0.000000
pause_time_max                             0.000000
pause_time_p

In [41]:
(
    X_train_transform
    .drop(columns='delete_insert_ratio_stddev')
    .to_pickle("./data/processed/X_train.pkl")
)